### Image Classification on the MNIST Dataset

In this chapter, I learn image classification Machine Learning algorithms, and how they work. Before anything, prepare the workspace.

In [20]:
import os
import shutil
from sklearn.datasets import fetch_openml

MB = 1024 * 1024
DATA_HOME = "../datasets"
NEW_DATASET_EVERYTIME = False

print("New dataset everytime:", NEW_DATASET_EVERYTIME)

if NEW_DATASET_EVERYTIME and os.path.exists(DATA_HOME):
    shutil.rmtree(DATA_HOME)
    print("Dataset directory removed")

    os.makedirs(DATA_HOME)
    print("Now downloading dataset")

if not NEW_DATASET_EVERYTIME:
    print("Reusing existing dataset")

MNIST = fetch_openml("mnist_784", as_frame=False, data_home=DATA_HOME)

data_size = MNIST.data.nbytes / MB
target_size = MNIST.target.nbytes / MB
total = data_size + target_size

print(f"\n{'Dataset size:':12} {data_size:4.2f} MB")
print(f"{'Target size:':12} {target_size:4.2f} MB")
print(f"{'Total size:':12} {total:4.2f} MB\n")

print("Done")

New dataset everytime: False
Reusing existing dataset

Dataset size: 418.70 MB
Target size: 0.53 MB
Total size:  419.24 MB

Done
